### Word Counting

In [ ]:
from tqdm.notebook import tqdm, trange
import re
def build_vocab(sentences, verbose=True):
    vocab = {}
    for i in trange(len(sentences)):
        for word in re.split(',| ', sentences[i]):
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

### Train+Val row data

In [ ]:
import numpy as np
with open('data/train/train_label.txt', 'r') as f:
    data_train = ['data/train/train_images/'+line.strip().strip('.') for line in f.readlines()]
with open('data/val/val_label.txt', 'r') as f:
    data_val = ['data/val/val_images/'+line.strip().strip('.') for line in f.readlines()]
data = data_train+data_val
train_cars = []
train_peoples = []
train_cars_path = []
train_peoples_path = []
train_cars_code = []
train_peoples_code = []
for line in data:
    if any(s.isdigit() for s in line):
        train_peoples.append(line.split('$')[-1])
        train_peoples_path.append(line.split('$')[0])
        train_peoples_code.append(line.split('$')[1])
    else:
        train_cars.append(line.split('$')[-1])
        train_cars_path.append(line.split('$')[0])
        train_cars_code.append(line.split('$')[1])

### Test row data

In [ ]:
import numpy as np
import pandas as pd
with open('data/test/test_text.txt', 'r') as f:
    data = [line.strip().strip('.') for line in f.readlines()]
test_peoples = [d for d in data if (' female' in d or ' woman' in d or ' male' in d or ' man' in d)]
test_cars = [d for d in data if not (' female' in d or ' woman' in d or ' male' in d or ' man' in d)]
# test_cars = data[:7611]
# test_peoples = data[7611:]

### Make train code (people)

In [ ]:
import pandas as pd
df = pd.DataFrame({'img_path':train_peoples_path})
df['code'] = -1
for i, code in enumerate(train_peoples_code):
    code = [int(j) for j in code.split(',')]
    new_code = []
    if code[0] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if code[1] == 1:
        new_code.extend([1,0,0])
    elif code[2] == 1:
        new_code.extend([0,1,0])
    elif code[3] == 1:
        new_code.extend([0,0,1])
    if code[4] == 1:
        new_code.extend([1,0,0])
    elif code[5] == 1:
        new_code.extend([0,1,0])
    elif code[6] == 1:
        new_code.extend([0,0,1])
    if code[7] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if code[8] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if code[9] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if code[10] == 1:
        new_code.extend([1,0,0])
    elif code[11] == 1:
        new_code.extend([0,1,0])
    else:
        new_code.extend([0,0,1])
    if code[12] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if code[13] == 1:
        new_code.extend([1,0])
    elif code[14] == 1:
        new_code.extend([0,1])
    if code[15] == 1:
        new_code.extend([1,0,0,0,0])
    elif code[16] == 1:
        new_code.extend([0,1,0,0,0])
    elif code[17] == 1:
        new_code.extend([0,0,1,0,0])
    elif code[18] == 1:
        new_code.extend([0,0,0,1,0])
    else:
        new_code.extend([0,0,0,0,1])
    if code[19] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if code[20] == 1:
        new_code.extend([1,0])
    else:
        new_code.extend([0,1])
    if sum(new_code)!=12:
        raise NameError('new_code is wrong')
    df.loc[i, 'code'] = str(new_code)
df.to_csv('data/train_val_peoples.csv', index=False)
df

In [ ]:
# # 0: female/woman
# # 1,2,3: elder, adult, youger
# # 4,5,6: face, sideway, back
# # 7: hat
# # 8: glasses
# # 9: handbag
# # 10,11:shoulderbag, backpack (could be all 0)
# # 12: holds objects in front
# # 13,14: short sleeves, long sleeves
# # 15,16,17,18: upper stripe, upper logo, upper plaid, upper splice (could be all 0)
# # 19: coat
# # 20: skirt or dress
# num = 0
# index = 6
# sentences = []
# for i, code in enumerate(train_peoples_code):
#     code_splits = [int(j) for j in code.split(',')]
#     cap = train_peoples[i]
#     # if code_splits[index] == 1 and ('body back to the camera' not in cap):
#     #     print(code_splits, train_peoples[i])
#     #     sentences.append(train_peoples[i])
#     #     num += 1
#     if code_splits[index] != 1 and ('body back to the camera' in cap):
#         print(code_splits, train_peoples[i])
#         num += 1
# print(num)
# vocab = build_vocab(sentences)
# print(f"There are {len(vocab)} unique words")
# vocab.keys()


### Make test code (people)

In [ ]:
test_codes = np.zeros((len(test_peoples), 30), dtype=np.int32)
# 0-1, 2-4, 5-7, 8-9, 10-11, 12-13, 14-16, 17-18, 19-20, 21-25, 26-27, 28-29
for idx, cap in enumerate(test_peoples):
    if ' female' in cap or ' woman' in cap:
        test_codes[idx, 0] = 1
    else:
        test_codes[idx, 1] = 1

    if 'aged 60 or older' in cap or 'over 60' in cap or 'at least 60 years old' in cap:
        test_codes[idx, 2] = 1
    elif 'between 18 and 60 years old' in cap or 'aged 18-60' in cap:
        test_codes[idx, 3] = 1
    elif 'is a minor' in cap or 'age 18 or younger' in cap or 'not an adult' in cap or 'less than 18 years old' in cap:
        test_codes[idx, 4] = 1

    if 'body facing the camera' in cap:
        test_codes[idx, 5] = 1
    elif 'standing sideways' in cap:
        test_codes[idx, 6] = 1
    elif 'body back to the camera' in cap:
        test_codes[idx, 7] = 1

    if 'wearning a hat' in cap:
        test_codes[idx, 8] = 1

    if 'glasses' in cap:
        test_codes[idx, 10] = 1

    if 'handbag' in cap:
        test_codes[idx, 12] = 1

    if 'shoulderbag' in cap or 'is carrying a shoulder bag' in cap or 'has a bag over the shoulder' in cap: #some captinos is wrong, code is right, num=2280
        test_codes[idx, 14] = 1
    elif 'backpack' in cap: #some captinos is wrong, code is right, num=636
        test_codes[idx, 15] = 1

    if 'holds objects in front' in cap: #some captinos is wrong, code is right, num=155
        test_codes[idx, 17] = 1

    if 'short sleeve' in cap: #some captinos is wrong, code is right, num=8092
        test_codes[idx, 19] = 1
    elif 'long sleeve' in cap: #some captinos is wrong, code is right, num=15026
        test_codes[idx, 20] = 1

    if 'upper stripe' in cap: #some captinos is wrong, code is right, num=677
        test_codes[idx, 21] = 1
    elif 'upper logo' in cap or 'logo on the upper' in cap: #some captinos is wrong, code is right, num=2214
        test_codes[idx, 22] = 1
    elif 'upper plaid' in cap or 'plaid on the upper' in cap: #some captinos is wrong, code is right, num=1520
        test_codes[idx, 23] = 1
    elif 'upper splice' in cap or 'splice on the upper' in cap: #some captinos is wrong, code is right, num=708
        test_codes[idx, 24] = 1

    if 'coat' in cap:
        test_codes[idx, 26] = 1

    if 'skirt or dress' in cap: #some captinos is wrong, code is right, num=887
        test_codes[idx, 28] = 1

code_str = [str(code) for code in test_codes]
df = pd.DataFrame({'prompt':test_peoples, 'code':code_str})
df.to_csv('data/test/test_people.csv', index=False)
df

### Unique words statistics

In [ ]:
vocab_train_cars = build_vocab(train_cars)
print(f"There are {len(vocab_train_cars)} unique words in train_cars")
vocab_train_peoples = build_vocab(train_peoples)
print(f"There are {len(vocab_train_peoples)} unique words in train_peoples")

In [ ]:
vocab_test_cars = build_vocab(test_cars)
print(f"There are {len(vocab_test_cars)} unique words in test_cars")
vocab_test_peoples = build_vocab(test_peoples)
print(f"There are {len(vocab_test_peoples)} unique words in test_peoples")

In [ ]:
assert vocab_train_cars.keys() == vocab_test_cars.keys()
assert vocab_train_peoples.keys() == vocab_test_peoples.keys()

### Car cap to code

In [ ]:
import pandas as pd
car_types = []
car_colors = []
car_brands = []
for cap in test_cars:
    s = cap.split()
    type_ = s[-1]
    if type_ not in car_types:
        car_types.append(type_)
    for c in s:
        if c!='is' and c!='a' and c!='an' and c.islower():
            color = c
            if color not in car_colors:
                car_colors.append(color)
    brand = cap.split(type_)[0].split(color)[-1].strip()
    if brand not in car_brands:
        car_brands.append(brand)

print('num_car_type:', len(car_types))
print('num_car_colors:', len(car_colors))
print('num_car_brands:', len(car_brands))
df1 = pd.DataFrame({'type_name':car_types})
df2 = pd.DataFrame({'color_name':car_colors})
df3 = pd.DataFrame({'brand_name':car_brands})
df1.to_csv('data/car_text/car_types.csv', index=False)
df2.to_csv('data/car_text/car_colors.csv', index=False)
df3.to_csv('data/car_text/car_brands.csv', index=False)

In [ ]:
import pandas as pd
df = pd.DataFrame({'img_path':train_cars_path})
df1 = pd.read_csv('data/car_text/car_types.csv')
df2 = pd.read_csv('data/car_text/car_colors.csv')
df3 = pd.read_csv('data/car_text/car_brands.csv')
types = df1['type_name'].tolist()
colors = df2['color_name'].tolist()
brands = df3['brand_name'].tolist()
df['type_name'] = np.nan
df['color_name'] = np.nan
df['brand_name'] = np.nan
df['type'] = np.nan
df['color'] = np.nan
df['brand'] = np.nan
for i, code in enumerate(train_cars):
    for t in types:
        if t in code:
            df.loc[i, 'type_name'] = t
            df.loc[i, 'type'] = types.index(t)
    for c in colors:
        if c in code:
            df.loc[i, 'color_name'] = c
            df.loc[i, 'color'] = colors.index(c)
    for b in brands:
        if len(b.split()) == 2:
            if b in code:
                df.loc[i, 'brand_name'] = b
                df.loc[i, 'brand'] = brands.index(b)
        elif b in code.split():
            df.loc[i, 'brand_name'] = b
            df.loc[i, 'brand'] = brands.index(b)
df.to_csv('data/train_val_cars.csv', index=False)
df

#### Test car

In [ ]:
import pandas as pd
df = pd.DataFrame({'prompt':test_cars})
df1 = pd.read_csv('data/car_text/car_types.csv')
df2 = pd.read_csv('data/car_text/car_colors.csv')
df3 = pd.read_csv('data/car_text/car_brands.csv')
types = df1['type_name'].tolist()
colors = df2['color_name'].tolist()
brands = df3['brand_name'].tolist()
df['type'] = np.nan
df['color'] = np.nan
df['brand'] = np.nan
for i, code in enumerate(test_cars):
    for t in types:
        if t in code.split():
            df.loc[i, 'type'] = types.index(t)
    for c in colors:
        if c in code.split():
            df.loc[i, 'color'] = colors.index(c)
    for b in brands:
        if len(b.split()) == 2:
            if b in code:
                df.loc[i, 'brand'] = brands.index(b)
        elif b in code.split():
            df.loc[i, 'brand'] = brands.index(b)

df.to_csv('data/test/test_car.csv', index=False)
df